In [2]:
!pip install --upgrade pip

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 4.6 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: pip
    Found existing installation: pip 24.1
    Uninstalling pip-24.1:
      Successfully uninstalled pip-24.1


In [3]:
!pip3 install unstructured libmagic python-magic python-magic-bin

  Using cached unstructured-0.14.9-py3-none-any.whl.metadata (28 kB)
  Using cached libmagic-1.0.tar.gz (3.7 kB)
  Preparing metadata (setup.py) ... done
  Using cached python_magic-0.4.27-py2.py3-none-any.whl.metadata (5.8 kB)
ERROR: Could not find a version that satisfies the requirement python-magic-bin (from versions: none)
ERROR: No matching distribution found for python-magic-bin


In [10]:
import os
import panel as pn
import param
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import DocArrayInMemorySearch
from langchain.chains import ConversationalRetrievalChain
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredURLLoader

# Set your OpenAI API key here
os.environ["OPENAI_API_KEY"] = ""

def load_url(url, chain_type, k):
 
    loader = UnstructuredURLLoader(urls=[url])
    documents = loader.load()


    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=150)
    docs = text_splitter.split_documents(documents)


    embeddings = OpenAIEmbeddings()


    db = DocArrayInMemorySearch.from_documents(docs, embeddings)


    retriever = db.as_retriever(search_type="similarity", search_kwargs={"k": k})


    qa = ConversationalRetrievalChain.from_llm(
        llm=ChatOpenAI(model_name='gpt-3.5-turbo', temperature=0),
        chain_type=chain_type,
        retriever=retriever,
        return_source_documents=True,
        return_generated_question=True,
    )
    return qa

class cbfs(param.Parameterized):
    chat_history = param.List([])
    answer = param.String("")
    db_query = param.String("")
    db_response = param.List([])
    
    def __init__(self, **params):
        super(cbfs, self).__init__(**params)
        self.panels = []
        self.loaded_url = "https://www.moneycontrol.com/news/business/tata-motors-mahindra-gain-certificates-for-production-linked-payouts-11281691.html"
        self.qa = load_url(self.loaded_url, "stuff", 4)
    
    def call_load_url(self, count):
        if count == 0 or url_input.value == '':  # init or no URL specified
            return pn.pane.Markdown(f"Loaded URL: {self.loaded_url}")
        else:
            self.loaded_url = url_input.value
            button_load.button_style = "outline"
            self.qa = load_url(self.loaded_url, "stuff", 4)
            button_load.button_style = "solid"
        self.clr_history()
        return pn.pane.Markdown(f"Loaded URL: {self.loaded_url}")

    def convchain(self, query):
        if not query:
            return pn.WidgetBox(pn.Row('User:', pn.pane.Markdown("", width=800)), scroll=True)
        result = self.qa({"question": query, "chat_history": self.chat_history})
        self.chat_history.extend([(query, result["answer"])])
        self.db_query = result["generated_question"]
        self.db_response = result["source_documents"]
        self.answer = result['answer'] 
        self.panels.extend([
            pn.Row('User:', pn.pane.Markdown(query, width=800)),
            pn.Row('ChatBot:', pn.pane.Markdown(self.answer, width=800, style={'background-color': '#F6F6F6'}, scroll=True))
        ])
        inp.value = ''  # clears loading indicator when cleared
        return pn.WidgetBox(*self.panels, scroll=True)

    @param.depends('db_query')
    def get_lquest(self):
        if not self.db_query:
            return pn.Column(
                pn.Row(pn.pane.Markdown(f"Last question to DB:", styles={'background-color': '#F6F6F6'})),
                pn.Row(pn.pane.Str("no DB accesses so far"))
            )
        return pn.Column(
            pn.Row(pn.pane.Markdown(f"DB query:", styles={'background-color': '#F6F6F6'})),
            pn.pane.Str(self.db_query)
        )

    @param.depends('db_response')
    def get_sources(self):
        if not self.db_response:
            return 
        rlist = [pn.Row(pn.pane.Markdown(f"Result of DB lookup:", styles={'background-color': '#F6F6F6'}))]
        for doc in self.db_response:
            rlist.append(pn.Row(pn.pane.Str(doc)))
        return pn.WidgetBox(*rlist, width=800, scroll=True)

    @param.depends('convchain', 'clr_history')
    def get_chats(self):
        if not self.chat_history:
            return pn.WidgetBox(pn.Row(pn.pane.Str("No History Yet")), width=600, scroll=True)
        rlist = [pn.Row(pn.pane.Markdown(f"Current Chat History variable", styles={'background-color': '#F6F6F6'}))]
        for exchange in self.chat_history:
            rlist.append(pn.Row(pn.pane.Str(exchange)))
        return pn.WidgetBox(*rlist, width=800, scroll=True)

    def clr_history(self, count=0):
        self.chat_history = []
        return 


pn.extension()


cb = cbfs()
url_input = pn.widgets.TextInput(placeholder='Enter URL here…')
button_load = pn.widgets.Button(name="Load URL", button_type='primary')
button_clearhistory = pn.widgets.Button(name="Clear History", button_type='warning')
inp = pn.widgets.TextInput(placeholder='Enter text here…')


button_clearhistory.on_click(cb.clr_history)
bound_button_load = pn.bind(cb.call_load_url, button_load.param.clicks)
conversation = pn.bind(cb.convchain, inp)

jpg_pane = pn.pane.Image('./img/convchain.jpg')


tab1 = pn.Column(
    pn.Row(inp),
    pn.layout.Divider(),
    pn.panel(conversation, loading_indicator=True, scroll=True),  # Ensure scroll=True for scrollable content
    pn.layout.Divider(),
)

tab2 = pn.Column(
    pn.panel(cb.get_lquest, width_policy='max', height_policy='max', scroll=True),  # Enable horizontal scrolling
    pn.layout.Divider(),
    pn.panel(cb.get_sources, width_policy='max', height_policy='max', scroll=True),  # Enable horizontal scrolling
)

tab3 = pn.Column(
    pn.panel(cb.get_chats, width_policy='max', height_policy='max', scroll=True),  # Enable horizontal scrolling
    pn.layout.Divider(),
)

tab4 = pn.Column(
    pn.Row(url_input, button_load, bound_button_load),
    pn.Row(button_clearhistory, pn.pane.Markdown("Clears chat history. Can use to start a new topic")),
    pn.layout.Divider(),
    pn.Row(jpg_pane.clone(width=400)),
)


dashboard = pn.Column(
    pn.Row(pn.pane.Markdown('# ChatWithYourData_Bot')),
    pn.Tabs(
        ('Conversation', tab1),
        ('Database', tab2),
        ('Chat History', tab3),
        ('Configure', tab4),
        tabs_location='above',
    ),
    sizing_mode='stretch_both',
)


dashboard.servable()


[nltk_data] Downloading package punkt to /Users/admin/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/admin/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
/Users/admin/anaconda3/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(
/Users/admin/anaconda3/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai pack

Column(sizing_mode='stretch_both')
    [0] Row
        [0] Markdown(str)
    [1] Tabs
        [0] Column
            [0] Row
                [0] TextInput(placeholder='Enter text here…')
            [1] Divider()
            [2] ParamFunction(function, _pane=WidgetBox, defer_load=False, loading_indicator=True)
            [3] Divider()
        [1] Column
            [0] ParamMethod(method, _pane=Column, defer_load=False, height_policy='max', width_policy='max')
            [1] Divider()
            [2] ParamMethod(method, _pane=Str, defer_load=False, height_policy='max', width_policy='max')
        [2] Column
            [0] ParamMethod(method, _pane=WidgetBox, defer_load=False, height_policy='max', width_policy='max')
            [1] Divider()
        [3] Column
            [0] Row
                [0] TextInput(placeholder='Enter URL here…')
                [1] Button(button_type='primary', name='Load URL')
                [2] ParamFunction(function, _pane=Markdown, defer_load=False)
            [1] Row
                [0] Button(button_type='warning', name='Clear History')
                [1] Markdown(str)
            [2] Divider()
            [3] Row
                [0] Image(str, width=400)

In [9]:
!pip install unstructured

  Using cached unstructured-0.14.9-py3-none-any.whl.metadata (28 kB)
  Using cached python_magic-0.4.27-py2.py3-none-any.whl.metadata (5.8 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 3.0 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
  Using cached charset_normalizer-3.3.2-cp311-cp311-macosx_11_0_arm64.whl.metadata (33 kB)
  Using cached dataclasses_json-0.6.7-py3-none-any.whl.metadata (25 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 3.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 431.4/431.4 kB 5.0 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 274.7/274.7 kB 6.3 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 5.0 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.0/41.0 kB 2.2 MB/s eta 0:00:00
Using cached dataclasses_json-0.6.7-py3-none-any.whl (28 kB)
Using cached charset_normalizer-3.3.2-cp311-cp311-macosx_11_0_arm64.whl (

In [13]:
import os
import requests
from bs4 import BeautifulSoup
import panel as pn
import param
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import DocArrayInMemorySearch
from langchain.chains import ConversationalRetrievalChain
from langchain.chat_models import ChatOpenAI
from langchain.schema import Document

os.environ["OPENAI_API_KEY"] = ""

def load_url_content(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    text = soup.get_text()
    return text

def load_url(url, chain_type, k):
    content = load_url_content(url)
    documents = [Document(page_content=content)]
    
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=150)
    docs = text_splitter.split_documents(documents)

    embeddings = OpenAIEmbeddings()

    db = DocArrayInMemorySearch.from_documents(docs, embeddings)

    retriever = db.as_retriever(search_type="similarity", search_kwargs={"k": k})

    qa = ConversationalRetrievalChain.from_llm(
        llm=ChatOpenAI(model_name='gpt-3.5-turbo', temperature=0),
        chain_type=chain_type,
        retriever=retriever,
        return_source_documents=True,
        return_generated_question=True,
    )
    return qa

class cbfs(param.Parameterized):
    chat_history = param.List([])
    answer = param.String("")
    db_query = param.String("")
    db_response = param.List([])
    
    def __init__(self, **params):
        super(cbfs, self).__init__(**params)
        self.panels = []
        self.loaded_url = "https://www.moneycontrol.com/news/business/tata-motors-mahindra-gain-certificates-for-production-linked-payouts-11281691.html"
        self.qa = load_url(self.loaded_url, "stuff", 4)
    
    def call_load_url(self, count):
        if count == 0 or url_input.value == '':  # init or no URL specified
            return pn.pane.Markdown(f"Loaded URL: {self.loaded_url}")
        else:
            self.loaded_url = url_input.value
            button_load.button_style = "outline"
            self.qa = load_url(self.loaded_url, "stuff", 4)
            button_load.button_style = "solid"
        self.clr_history()
        return pn.pane.Markdown(f"Loaded URL: {self.loaded_url}")

    def convchain(self, query):
        if not query:
            return pn.WidgetBox(pn.Row('User:', pn.pane.Markdown("", width=800)), scroll=True)
        result = self.qa({"question": query, "chat_history": self.chat_history})
        self.chat_history.extend([(query, result["answer"])])
        self.db_query = result["generated_question"]
        self.db_response = result["source_documents"]
        self.answer = result['answer'] 
        self.panels.extend([
            pn.Row('User:', pn.pane.Markdown(query, width=800)),
            pn.Row('ChatBot:', pn.pane.Markdown(self.answer, width=800, style={'background-color': '#F6F6F6'}, scroll=True))
        ])
        inp.value = ''  # clears loading indicator when cleared
        return pn.WidgetBox(*self.panels, scroll=True)

    @param.depends('db_query')
    def get_lquest(self):
        if not self.db_query:
            return pn.Column(
                pn.Row(pn.pane.Markdown(f"Last question to DB:", styles={'background-color': '#F6F6F6'})),
                pn.Row(pn.pane.Str("no DB accesses so far"))
            )
        return pn.Column(
            pn.Row(pn.pane.Markdown(f"DB query:", styles={'background-color': '#F6F6F6'})),
            pn.pane.Str(self.db_query)
        )

    @param.depends('db_response')
    def get_sources(self):
        if not self.db_response:
            return 
        rlist = [pn.Row(pn.pane.Markdown(f"Result of DB lookup:", styles={'background-color': '#F6F6F6'}))]
        for doc in self.db_response:
            rlist.append(pn.Row(pn.pane.Str(doc)))
        return pn.WidgetBox(*rlist, width=800, scroll=True)

    @param.depends('convchain', 'clr_history')
    def get_chats(self):
        if not self.chat_history:
            return pn.WidgetBox(pn.Row(pn.pane.Str("No History Yet")), width=600, scroll=True)
        rlist = [pn.Row(pn.pane.Markdown(f"Current Chat History variable", styles={'background-color': '#F6F6F6'}))]
        for exchange in self.chat_history:
            rlist.append(pn.Row(pn.pane.Str(exchange)))
        return pn.WidgetBox(*rlist, width=800, scroll=True)

    def clr_history(self, count=0):
        self.chat_history = []
        return 

pn.extension()

cb = cbfs()
url_input = pn.widgets.TextInput(placeholder='Enter URL here…')
button_load = pn.widgets.Button(name="Load URL", button_type='primary')
button_clearhistory = pn.widgets.Button(name="Clear History", button_type='warning')
inp = pn.widgets.TextInput(placeholder='Enter text here…')

button_clearhistory.on_click(cb.clr_history)
bound_button_load = pn.bind(cb.call_load_url, button_load.param.clicks)
conversation = pn.bind(cb.convchain, inp)

jpg_pane = pn.pane.Image('./img/convchain.jpg')

tab1 = pn.Column(
    pn.Row(inp),
    pn.layout.Divider(),
    pn.panel(conversation, loading_indicator=True, scroll=True),  # Ensure scroll=True for scrollable content
    pn.layout.Divider(),
)

tab2 = pn.Column(
    pn.panel(cb.get_lquest, width_policy='max', height_policy='max', scroll=True),  # Enable horizontal scrolling
    pn.layout.Divider(),
    pn.panel(cb.get_sources, width_policy='max', height_policy='max', scroll=True),  # Enable horizontal scrolling
)

tab3 = pn.Column(
    pn.panel(cb.get_chats, width_policy='max', height_policy='max', scroll=True),  # Enable horizontal scrolling
    pn.layout.Divider(),
)

tab4 = pn.Column(
    pn.Row(url_input, button_load, bound_button_load),
    pn.Row(button_clearhistory, pn.pane.Markdown("Clears chat history. Can use to start a new topic")),
    pn.layout.Divider(),
    pn.Row(jpg_pane.clone(width=400)),
)

dashboard = pn.Column(
    pn.Row(pn.pane.Markdown('# ChatWithYourData_Bot')),
    pn.Tabs(
        ('Conversation', tab1),
        ('Database', tab2),
        ('Chat History', tab3),
        ('Configure', tab4),
        tabs_location='above',
    ),
    sizing_mode='stretch_both',
)

dashboard.servable()


Column(sizing_mode='stretch_both')
    [0] Row
        [0] Markdown(str)
    [1] Tabs
        [0] Column
            [0] Row
                [0] TextInput(placeholder='Enter text here…')
            [1] Divider()
            [2] ParamFunction(function, _pane=WidgetBox, defer_load=False, loading_indicator=True)
            [3] Divider()
        [1] Column
            [0] ParamMethod(method, _pane=Column, defer_load=False, height_policy='max', width_policy='max')
            [1] Divider()
            [2] ParamMethod(method, _pane=Str, defer_load=False, height_policy='max', width_policy='max')
        [2] Column
            [0] ParamMethod(method, _pane=WidgetBox, defer_load=False, height_policy='max', width_policy='max')
            [1] Divider()
        [3] Column
            [0] Row
                [0] TextInput(placeholder='Enter URL here…')
                [1] Button(button_type='primary', name='Load URL')
                [2] ParamFunction(function, _pane=Markdown, defer_load=False)
            [1] Row
                [0] Button(button_type='warning', name='Clear History')
                [1] Markdown(str)
            [2] Divider()
            [3] Row
                [0] Image(str, width=400)

In [14]:
import os
import requests
from bs4 import BeautifulSoup
import panel as pn
import param
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import DocArrayInMemorySearch
from langchain.chains import ConversationalRetrievalChain
from langchain.chat_models import ChatOpenAI
from langchain.schema import Document

os.environ["OPENAI_API_KEY"] = ""

def load_url_content(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    text = soup.get_text()
    return text

def load_urls(urls, chain_type, k):
    documents = []
    for url in urls:
        content = load_url_content(url)
        documents.append(Document(page_content=content, metadata={"source": url}))
    
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=150)
    docs = text_splitter.split_documents(documents)

    embeddings = OpenAIEmbeddings()

    db = DocArrayInMemorySearch.from_documents(docs, embeddings)

    retriever = db.as_retriever(search_type="similarity", search_kwargs={"k": k})

    qa = ConversationalRetrievalChain.from_llm(
        llm=ChatOpenAI(model_name='gpt-3.5-turbo', temperature=0),
        chain_type=chain_type,
        retriever=retriever,
        return_source_documents=True,
        return_generated_question=True,
    )
    return qa

class cbfs(param.Parameterized):
    chat_history = param.List([])
    answer = param.String("")
    db_query = param.String("")
    db_response = param.List([])

    def __init__(self, **params):
        super(cbfs, self).__init__(**params)
        self.panels = []
        self.loaded_urls = [
            "https://www.moneycontrol.com/news/business/tata-motors-mahindra-gain-certificates-for-production-linked-payouts-11281691.html"
        ]
        self.qa = load_urls(self.loaded_urls, "stuff", 4)

    def call_load_urls(self, count):
        if count == 0 or url_input.value == '':  # init or no URLs specified
            return pn.pane.Markdown(f"Loaded URLs: {self.loaded_urls}")
        else:
            self.loaded_urls = url_input.value.split(",")
            button_load.button_style = "outline"
            self.qa = load_urls(self.loaded_urls, "stuff", 4)
            button_load.button_style = "solid"
        self.clr_history()
        return pn.pane.Markdown(f"Loaded URLs: {self.loaded_urls}")

    def convchain(self, query):
        if not query:
            return pn.WidgetBox(pn.Row('User:', pn.pane.Markdown("", width=800)), scroll=True)
        result = self.qa({"question": query, "chat_history": self.chat_history})
        self.chat_history.extend([(query, result["answer"])])
        self.db_query = result["generated_question"]
        self.db_response = result["source_documents"]
        self.answer = result['answer']
        self.panels.extend([
            pn.Row('User:', pn.pane.Markdown(query, width=800)),
            pn.Row('ChatBot:', pn.pane.Markdown(self.answer, width=800, style={'background-color': '#F6F6F6'}, scroll=True))
        ])
        inp.value = ''  # clears loading indicator when cleared
        return pn.WidgetBox(*self.panels, scroll=True)

    @param.depends('db_query')
    def get_lquest(self):
        if not self.db_query:
            return pn.Column(
                pn.Row(pn.pane.Markdown(f"Last question to DB:", styles={'background-color': '#F6F6F6'})),
                pn.Row(pn.pane.Str("no DB accesses so far"))
            )
        return pn.Column(
            pn.Row(pn.pane.Markdown(f"DB query:", styles={'background-color': '#F6F6F6'})),
            pn.pane.Str(self.db_query)
        )

    @param.depends('db_response')
    def get_sources(self):
        if not self.db_response:
            return 
        rlist = [pn.Row(pn.pane.Markdown(f"Result of DB lookup:", styles={'background-color': '#F6F6F6'}))]
        for doc in self.db_response:
            rlist.append(pn.Row(pn.pane.Str(doc)))
        return pn.WidgetBox(*rlist, width=800, scroll=True)

    @param.depends('convchain', 'clr_history')
    def get_chats(self):
        if not self.chat_history:
            return pn.WidgetBox(pn.Row(pn.pane.Str("No History Yet")), width=600, scroll=True)
        rlist = [pn.Row(pn.pane.Markdown(f"Current Chat History variable", styles={'background-color': '#F6F6F6'}))]
        for exchange in self.chat_history:
            rlist.append(pn.Row(pn.pane.Str(exchange)))
        return pn.WidgetBox(*rlist, width=800, scroll=True)

    def clr_history(self, count=0):
        self.chat_history = []
        return 

pn.extension()

cb = cbfs()
url_input = pn.widgets.TextInput(placeholder='Enter URLs here (comma-separated)…')
button_load = pn.widgets.Button(name="Load URLs", button_type='primary')
button_clearhistory = pn.widgets.Button(name="Clear History", button_type='warning')
inp = pn.widgets.TextInput(placeholder='Enter text here…')

button_clearhistory.on_click(cb.clr_history)
bound_button_load = pn.bind(cb.call_load_urls, button_load.param.clicks)
conversation = pn.bind(cb.convchain, inp)

jpg_pane = pn.pane.Image('./img/convchain.jpg')

tab1 = pn.Column(
    pn.Row(inp),
    pn.layout.Divider(),
    pn.panel(conversation, loading_indicator=True, scroll=True),  # Ensure scroll=True for scrollable content
    pn.layout.Divider(),
)

tab2 = pn.Column(
    pn.panel(cb.get_lquest, width_policy='max', height_policy='max', scroll=True),  # Enable horizontal scrolling
    pn.layout.Divider(),
    pn.panel(cb.get_sources, width_policy='max', height_policy='max', scroll=True),  # Enable horizontal scrolling
)

tab3 = pn.Column(
    pn.panel(cb.get_chats, width_policy='max', height_policy='max', scroll=True),  # Enable horizontal scrolling
    pn.layout.Divider(),
)

tab4 = pn.Column(
    pn.Row(url_input, button_load, bound_button_load),
    pn.Row(button_clearhistory, pn.pane.Markdown("Clears chat history. Can use to start a new topic")),
    pn.layout.Divider(),
    pn.Row(jpg_pane.clone(width=400)),
)


dashboard = pn.Column(
    pn.Row(pn.pane.Markdown('# ChatWithYourData_Bot')),
    pn.Tabs(
        ('Conversation', tab1),
        ('Database', tab2),
        ('Chat History', tab3),
        ('Configure', tab4),
        tabs_location='above',
    ),
    sizing_mode='stretch_both',
)


dashboard.servable()


Column(sizing_mode='stretch_both')
    [0] Row
        [0] Markdown(str)
    [1] Tabs
        [0] Column
            [0] Row
                [0] TextInput(placeholder='Enter text here…')
            [1] Divider()
            [2] ParamFunction(function, _pane=WidgetBox, defer_load=False, loading_indicator=True)
            [3] Divider()
        [1] Column
            [0] ParamMethod(method, _pane=Column, defer_load=False, height_policy='max', width_policy='max')
            [1] Divider()
            [2] ParamMethod(method, _pane=Str, defer_load=False, height_policy='max', width_policy='max')
        [2] Column
            [0] ParamMethod(method, _pane=WidgetBox, defer_load=False, height_policy='max', width_policy='max')
            [1] Divider()
        [3] Column
            [0] Row
                [0] TextInput(placeholder='Enter URLs here (...)
                [1] Button(button_type='primary', name='Load URLs')
                [2] ParamFunction(function, _pane=Markdown, defer_load=False)
            [1] Row
                [0] Button(button_type='warning', name='Clear History')
                [1] Markdown(str)
            [2] Divider()
            [3] Row
                [0] Image(str, width=400)